# Puviyan Soil Detection - Enhanced Training v5.0

GPU-accelerated training for soil classification with TFLite export.

**Setup:** Runtime > Change runtime type > GPU

In [ ]:
# Setup Environment
print('Setting up environment...')
!pip install -q tensorflow matplotlib numpy tqdm

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import files

# Check GPU
print('GPU Available:', len(tf.config.list_physical_devices('GPU')) > 0)

In [ ]:
# Generate synthetic soil data
def create_data(samples=5000):
    X = np.random.randint(0, 255, (samples, 224, 224, 3), dtype=np.uint8)
    y = np.random.randint(0, 8, samples)
    return X, y

X_train, y_train = create_data(4000)
X_val, y_val = create_data(1000)
print(f'Data ready: {X_train.shape}, {X_val.shape}')

In [ ]:
# Create enhanced model
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.SeparableConv2D(64, 3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(8, activation='softmax')
])

model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Train model
print('Training started...')
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    verbose=1
)

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save and download
with open('soil_model.tflite', 'wb') as f:
    f.write(tflite_model)

print('Model saved! Size:', len(tflite_model) // 1024, 'KB')
files.download('soil_model.tflite')